In [ ]:
import os
os.environ['PYDEVD_DISABLE_FILE_VALIDATION'] = '1'

# Clean previous installations
!rm -rf Real-ESRGAN
!pip uninstall -y torch torchvision basicsr facexlib gfpgan

# Install compatible dependencies
!pip install torch==1.13.1 torchvision==0.13.1
!pip install numpy==1.23.5

# Clone and set up Real-ESRGAN
!git clone https://github.com/xinntao/Real-ESRGAN.git
%cd /content/Real-ESRGAN

# FIX: Install requirements BEFORE setup
!pip install -r requirements.txt
!pip install basicsr==1.4.2 facexlib==0.3.0 gfpgan==1.3.8
!pip install -U opencv-python opencv-contrib-python

# FIX: Use standard install instead of develop
!python setup.py install

# Download models
!wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P experiments/pretrained_models
!wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.2.4/RealESRGAN_x4plus_anime_6B.pth -P experiments/pretrained_models

# Patch broken import
!sed -i "s/from torchvision.transforms.functional_tensor import/from torchvision.transforms.functional import/g" /usr/local/lib/python3.*/dist-packages/basicsr/data/degradations.py

# Clone and set up GFPGAN
%cd /content
!git clone https://github.com/TencentARC/GFPGAN.git
%cd /content/GFPGAN

# FIX: Install requirements BEFORE setup
!pip install -r requirements.txt
!python setup.py install

In [ ]:
from google.colab import files
import subprocess

# Upload the .insv video
uploaded = files.upload()
input_video = next(iter(uploaded))

# Extract frame rate
result = subprocess.run(['ffprobe', '-v', 'error', '-select_streams', 'v:0',
                         '-show_entries', 'stream=r_frame_rate',
                         '-of', 'default=noprint_wrappers=1:nokey=1',
                         input_video],
                        stdout=subprocess.PIPE, text=True)
frame_rate = result.stdout.strip()

# Create directory for frames
!mkdir -p /content/frames

# Extract frames
!ffmpeg -i {input_video} /content/frames/%04d.png -hide_banner -loglevel error

# Extract audio
!ffmpeg -i {input_video} -vn -acodec copy /content/audio.aac -hide_banner -loglevel error

import cv2
import numpy as np
import os
import glob
import subprocess
import sys
from pathlib import Path

# Tuning parameters
USE_REALESRGAN_FACE_ENHANCE = True
GFPGAN_WEIGHT = 0.2
BLUR_KERNEL_SIZE = 151
BLENDING_ALPHA = 0.3

# Set absolute paths
BASE_DIR = Path("/content")
REALESRGAN_DIR = BASE_DIR / "Real-ESRGAN"
GFPGAN_DIR = BASE_DIR / "GFPGAN"

# Create output directories
output_dir = REALESRGAN_DIR / "results"
(output_dir / "upscaled").mkdir(parents=True, exist_ok=True)
(output_dir / "gfpgan_output" / "restored_imgs").mkdir(parents=True, exist_ok=True)
(output_dir / "final").mkdir(parents=True, exist_ok=True)

frame_list = sorted(glob.glob(str(BASE_DIR / "frames" / "*.png")))
print(f"Found {len(frame_list)} frames to process")

for frame_path in frame_list:
    frame_path = Path(frame_path)
    frame_name = frame_path.name
    print(f"\nProcessing {frame_name}...")

    # ======================================================================
    # 1. REAL-ESRGAN UPSAMPLING
    # ======================================================================
    upscaled_filename = frame_name.replace('.png', '_upscaled.png')
    upscaled_path = output_dir / "upscaled" / upscaled_filename

    if not upscaled_path.exists():
        face_enhance_flag = "--face_enhance" if USE_REALESRGAN_FACE_ENHANCE else ""
        cmd = [
            sys.executable, str(REALESRGAN_DIR / "inference_realesrgan.py"),
            "-n", "RealESRGAN_x4plus.pth",
            "-i", str(frame_path),
            "--outscale", "4",
            "--output", str(output_dir / "upscaled"),
            "--suffix", "upscaled"
        ]
        if face_enhance_flag:
            cmd.append(face_enhance_flag)

        print("  Running RealESRGAN...")

        # Set PYTHONPATH to include both modules
        env = os.environ.copy()
        env["PYTHONPATH"] = f"{REALESRGAN_DIR}:{GFPGAN_DIR}:{env.get('PYTHONPATH', '')}"

        result = subprocess.run(cmd, env=env, capture_output=True, text=True)

        if result.returncode != 0:
            print(f"  ❗ RealESRGAN failed for {frame_name}")
            print(f"  Error: {result.stderr[:500]}")
            continue

        # Verify output
        if not upscaled_path.exists():
            # Try to find any output file
            possible_files = list((output_dir / "upscaled").glob("*.png"))
            if possible_files:
                actual_path = possible_files[0]
                print(f"  Found alternative output: {actual_path}")
                os.rename(str(actual_path), str(upscaled_path))
            else:
                print(f"  ❗ No output created in {output_dir / 'upscaled'}")
                continue

    # ======================================================================
    # 2. GFPGAN ENHANCEMENT
    # ======================================================================
    gfpgan_path = output_dir / "gfpgan_output" / "restored_imgs" / upscaled_filename

    if not gfpgan_path.exists():
        cmd = [
            sys.executable, str(GFPGAN_DIR / "inference_gfpgan.py"),
            "-i", str(upscaled_path),
            "-o", str(output_dir / "gfpgan_output"),
            "-v", "1.4",
            "-s", "1",
            "--weight", str(GFPGAN_WEIGHT),
            "--bg_upsampler", "realesrgan"
        ]
        print("  Running GFPGAN...")

        # Set PYTHONPATH to include both modules
        env = os.environ.copy()
        env["PYTHONPATH"] = f"{REALESRGAN_DIR}:{GFPGAN_DIR}:{env.get('PYTHONPATH', '')}"

        result = subprocess.run(cmd, env=env, capture_output=True, text=True)

        if result.returncode != 0:
            print(f"  ❗ GFPGAN failed: {result.stderr[:500]}")
            # Try to find GFPGAN output in alternative location
            possible_gfpgan = list((output_dir / "gfpgan_output").rglob("*.png"))
            if possible_gfpgan:
                actual_path = possible_gfpgan[0]
                print(f"  Found GFPGAN output at: {actual_path}")
                gfpgan_path.parent.mkdir(parents=True, exist_ok=True)
                os.rename(str(actual_path), str(gfpgan_path))

    # ======================================================================
    # 3. PROCESS IMAGES
    # ======================================================================
    original_upscaled = cv2.imread(str(upscaled_path))
    if original_upscaled is None:
        print(f"  ❗ Failed to load upscaled image")
        continue

    # Use GFPGAN output if available, otherwise use original
    if gfpgan_path.exists():
        enhanced = cv2.imread(str(gfpgan_path))
    else:
        enhanced = original_upscaled.copy()

    # Resize if needed
    if enhanced.shape != original_upscaled.shape:
        enhanced = cv2.resize(enhanced, (original_upscaled.shape[1], original_upscaled.shape[0]))

    # ======================================================================
    # 4. FACE DETECTION AND BLENDING
    # ======================================================================
    print("  Blending results...")
    balanced_result = original_upscaled.copy()
    gray = cv2.cvtColor(original_upscaled, cv2.COLOR_BGR2GRAY)

    # Initialize face detector
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, 1.1, 4, minSize=(50, 50))

    if len(faces) > 0:
        print(f"  Detected {len(faces)} faces")
        mask = np.zeros(gray.shape[:2], dtype=np.float32)

        for (x, y, w, h) in faces:
            # Expand face area for blending
            y_exp = max(0, y - int(h * 0.25))
            x_exp = max(0, x - int(w * 0.25))
            h_exp = min(gray.shape[0] - y_exp, int(h * 1.5))
            w_exp = min(gray.shape[1] - x_exp, int(w * 1.5))

            # Create elliptical mask
            center = (x_exp + w_exp//2, y_exp + h_exp//2)
            axes = (int(w_exp*0.5), int(h_exp*0.5))
            cv2.ellipse(mask, center, axes, 0, 0, 360, 1.0, -1)

        # Smooth mask
        mask = cv2.GaussianBlur(mask, (BLUR_KERNEL_SIZE, BLUR_KERNEL_SIZE), 0)
        mask = np.clip(mask, 0, 1)
        mask_3d = np.dstack([mask, mask, mask])

        # Blend images
        original_float = original_upscaled.astype(np.float32)
        enhanced_float = enhanced.astype(np.float32)
        blended = original_float * (1 - mask_3d * BLENDING_ALPHA) + enhanced_float * (mask_3d * BLENDING_ALPHA)
        balanced_result = np.clip(blended, 0, 255).astype(np.uint8)
    else:
        print("  No faces detected - applying global blend")
        balanced_result = cv2.addWeighted(original_upscaled, 1 - BLENDING_ALPHA, enhanced, BLENDING_ALPHA, 0)

    # ======================================================================
    # 5. POST-PROCESSING AND SAVING
    # ======================================================================
    print("  Final processing...")
    # CLAHE for contrast enhancement
    lab = cv2.cvtColor(balanced_result, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=1.0, tileGridSize=(8,8))
    l = clahe.apply(l)
    balanced_result = cv2.merge((l, a, b))
    balanced_result = cv2.cvtColor(balanced_result, cv2.COLOR_LAB2BGR)

    # Gentle noise reduction
    balanced_result = cv2.bilateralFilter(balanced_result, 5, 25, 25)

    # Save final frame
    final_path = output_dir / "final" / frame_name
    cv2.imwrite(str(final_path), balanced_result)
    print(f"  ✅ Saved enhanced frame: {final_path}")

print("\nProcessing complete!")


In [ ]:
# ===================================================================
# VIDEO RECONSTRUCTION
# ===================================================================
print("\nReconstructing video...")
final_video = "enhanced_video.mp4"
!ffmpeg -framerate {frame_rate} -i /content/Real-ESRGAN/results/final/%04d.png \
        -i /content/audio.aac \
        -c:v libx264 -preset slow -crf 18 \
        -c:a aac -b:a 192k \
        -pix_fmt yuv420p {final_video} -hide_banner -loglevel error

print("Final video saved as enhanced_video.mp4")
files.download(final_video)